## DV2599 Assignment 2
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [5]:
# Import packages
import pandas as pd
import statistics as stat
import numpy as np
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Load dataset
df = pd.read_csv('spambase/spambase.data', delimiter=',', header=None)

# Add headers to columns to increase clarity and comprehensibility
headers = []
with open('spambase/spambase.names', 'r') as file:
    N = [line for line in file]
    headers = [x[:x.find(':')] for x in N[33:]]

headers.append("spam")  # Spam column is last, 1 = spam, 0 = not spam
df.columns = headers

# Split data into x and y
y = df['spam']
x = df.drop(columns='spam')

display(df)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


#### 0. Pre-process

In [6]:
# Initialize standard scaler
scaler = StandardScaler()

# Fit and transform train set
x_scaler = scaler.fit_transform(x)

# Turn back into pandas dataframe
x_scaled = pd.DataFrame(x_scaler, columns=x.columns, index=x.index)

# Describe
x_scaled.describe().round(2)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,...,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00
mean,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,...,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.51,-0.16,-0.33,-0.31,-0.10,-0.13,-0.26,-0.47
25%,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.51,-0.16,-0.33,-0.31,-0.10,-0.11,-0.24,-0.41
50%,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.27,-0.16,-0.33,-0.31,-0.10,-0.09,-0.19,-0.31
75%,-0.34,-0.17,0.28,-0.05,0.10,-0.35,-0.29,-0.26,-0.32,-0.12,...,-0.11,-0.16,0.18,-0.16,0.06,-0.10,-0.10,-0.05,-0.05,-0.03
max,14.53,10.90,9.56,30.64,14.41,21.13,18.28,27.44,18.56,27.83,...,34.89,17.85,35.56,37.15,39.49,24.11,46.09,34.59,50.99,25.66


#### 1. Stratified ten-fold cross-validation

In [7]:
# Initialize ten-fold cross-validation
skf = StratifiedKFold(n_splits=10)
score_types = ('fit_time', 'accuracy_score', 'f_score')

# Initialize models
rf = RandomForestClassifier()
lr = LogisticRegression()
sv = SVC()

models = [("Random Forest", rf), ("Logistic Regression", lr), ("SVC", sv)]
model_data = defaultdict(dict)

# Stratified K-fold for each model
for model_name, model in models:
    cv_accuracy = cross_validate(model, x_scaled, y, cv=skf, scoring="accuracy")
    cv_f1 = cross_validate(model, x_scaled, y, cv=skf, scoring="f1")
    model_data[model_name][score_types[0]] = cv_accuracy.get('fit_time')
    model_data[model_name][score_types[1]] = cv_accuracy.get('test_score')
    model_data[model_name][score_types[2]] = cv_f1.get('test_score')

#### 2. Results

In [8]:
column_1 = {"Fold":[1,2,3,4,5,6,7,8,9,10,"avg", "stdev"]}

### Create accuracy, f1 and time dataframes
df_acc = pd.DataFrame(column_1)
df_f1 = pd.DataFrame(column_1)
df_time = pd.DataFrame(column_1)

### Add data to dataframes
for model_name in model_data:
    for score_id, df in zip(score_types, (df_time, df_acc, df_f1)):
        score = model_data[model_name][score_id]
        df[model_name] = np.append(score, [stat.mean(score), stat.stdev(score)])

### Save tables
df_time.round(4).to_csv(f"df_time.csv", index=False)
df_acc.round(4).to_csv(f"df_acc.csv", index=False)
df_f1.round(4).to_csv(f"df_f.csv", index=False)

### Display
print("Time")
display(df_time)

print("Accuracy")
display(df_acc)

print("F1")
display(df_f1)

Time


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.557465,0.027693,0.189797
1,2,0.572153,0.016999,0.144508
2,3,0.534086,0.017016,0.142510
3,4,0.548128,0.015000,0.158283
4,5,0.534313,0.015610,0.158638
5,6,0.536556,0.017341,0.151026
6,7,0.540549,0.014997,0.150433
7,8,0.542170,0.013999,0.160125
8,9,0.532091,0.014286,0.150525
9,10,0.539089,0.013999,0.141510


Accuracy


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.950108,0.930586,0.924078
1,2,0.947826,0.921739,0.941304
2,3,0.934783,0.919565,0.934783
3,4,0.947826,0.939130,0.936957
4,5,0.954348,0.923913,0.947826
5,6,0.956522,0.930435,0.952174
6,7,0.969565,0.952174,0.956522
7,8,0.976087,0.926087,0.941304
8,9,0.891304,0.854348,0.891304
9,10,0.863043,0.871739,0.869565


F1


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.935211,0.908046,0.902507
1,2,0.935933,0.897727,0.925208
2,3,0.913793,0.896359,0.915730
3,4,0.935211,0.922222,0.917847
4,5,0.944134,0.902507,0.932584
5,6,0.947945,0.914439,0.939891
6,7,0.960674,0.936782,0.941860
7,8,0.963585,0.904494,0.923513
8,9,0.861619,0.820375,0.861111
9,10,0.821229,0.829971,0.821429


#### 3. Friedman test

In [9]:
friedman_scores = {}

### Perform Friedman test for each dataframe
for i, df in enumerate((df_time, df_acc, df_f1)):
    ### Remove last two rows and fold column
    df_friedman_score = df.iloc[:-2]
    df_rank = df_friedman_score.drop(columns='Fold')

    ### Rank
    df_rank = df_rank.rank(1, 'max', ascending=not i)

    ### Combine ranks and score values
    df_friedman_score = df_friedman_score.rename(columns={'Fold': 'Data set'})
    for column in df_rank.columns:
        combines = [f"{round(x,4)} ({int(y)})" for x, y in zip(df_friedman_score[column], df_rank[column])]
        df_friedman_score[column] = combines

    ### Add avg rank row
    df_rank_avg = df_rank.mean()
    df_friedman_score.loc[len(df_friedman_score)] = ['avg rank', *[df_rank_avg.iloc[x] for x in range(len(df_rank_avg))]] 

    ### Save friedman table
    df_friedman_score.to_csv(f"friedman_{score_types[i]}.csv", index=False)
    friedman_scores[score_types[i]] = df_friedman_score

    print(f"{score_types[i]}:")
    display(df_friedman_score)

fit_time:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.5575 (3),0.0277 (1),0.1898 (2)
1,2,0.5722 (3),0.017 (1),0.1445 (2)
2,3,0.5341 (3),0.017 (1),0.1425 (2)
3,4,0.5481 (3),0.015 (1),0.1583 (2)
4,5,0.5343 (3),0.0156 (1),0.1586 (2)
5,6,0.5366 (3),0.0173 (1),0.151 (2)
6,7,0.5405 (3),0.015 (1),0.1504 (2)
7,8,0.5422 (3),0.014 (1),0.1601 (2)
8,9,0.5321 (3),0.0143 (1),0.1505 (2)
9,10,0.5391 (3),0.014 (1),0.1415 (2)


accuracy_score:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.9501 (1),0.9306 (2),0.9241 (3)
1,2,0.9478 (1),0.9217 (3),0.9413 (2)
2,3,0.9348 (2),0.9196 (3),0.9348 (2)
3,4,0.9478 (1),0.9391 (2),0.937 (3)
4,5,0.9543 (1),0.9239 (3),0.9478 (2)
5,6,0.9565 (1),0.9304 (3),0.9522 (2)
6,7,0.9696 (1),0.9522 (3),0.9565 (2)
7,8,0.9761 (1),0.9261 (3),0.9413 (2)
8,9,0.8913 (2),0.8543 (3),0.8913 (2)
9,10,0.863 (3),0.8717 (1),0.8696 (2)


f_score:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.9352 (1),0.908 (2),0.9025 (3)
1,2,0.9359 (1),0.8977 (3),0.9252 (2)
2,3,0.9138 (2),0.8964 (3),0.9157 (1)
3,4,0.9352 (1),0.9222 (2),0.9178 (3)
4,5,0.9441 (1),0.9025 (3),0.9326 (2)
5,6,0.9479 (1),0.9144 (3),0.9399 (2)
6,7,0.9607 (1),0.9368 (3),0.9419 (2)
7,8,0.9636 (1),0.9045 (3),0.9235 (2)
8,9,0.8616 (1),0.8204 (3),0.8611 (2)
9,10,0.8212 (3),0.83 (1),0.8214 (2)


#### 5.1 Friedman values

In [18]:
n = 10
k = len(models)
critical_value = 6.20
post_hoc = []
print(f"Critical value = {critical_value}\n")

### R_mean value
R_mean = (k+1)/2

for i, df_name in enumerate(friedman_scores):
    df = friedman_scores.get(df_name)
    print(f"{df_name}:")

    ### R-mean
    R_values = []
    df_copy = df.iloc[:-1].drop(columns='Data set')
    for row in df_copy.to_numpy():
        for value in row:
            R_values.append(int(value[-2]))

    R_mean = stat.fmean(R_values)
    print("R mean =", R_mean)

    ### Sum of squared differences avg
    avg = list(df.iloc[-1])[1:]
    
    sum1 = n*sum([(x-R_mean)**2 for x in avg])
    print(f"sum1 = {round(sum1,4)}")

    ### Sum of squared differences all
    sum2 = 0
    for value in R_values:
        sum2 += (value-R_mean)**2
    sum2 *= 1/(n*(k-1))
    
    ### Calculate ratio
    f_stat = sum1/sum2
    if f_stat > critical_value:
        post_hoc.append(df_name)

    print(f"sum2= {round(sum2,4)}")
    print(f"Friedman statistic = {round(f_stat,4)}")
    print()


Critical value = 6.2

fit_time:
R mean = 2.0
sum1 = 20.0
sum2= 1.0
Friedman statistic = 20.0

accuracy_score:
R mean = 2.066666666666667
sum1 = 7.4667
sum2= 0.8933
Friedman statistic = 8.3582

f_score:
R mean = 2.0
sum1 = 8.6
sum2= 1.0
Friedman statistic = 8.6



##### Nemenyi test

In [7]:
### Critical distance
cd = round(2.343*(k*(k+1)/(6*n))**0.5, 4)
print(f"Critical distance = {cd}")
for i, df_name in enumerate(post_hoc):
    df = friedman_scores.get(df_name)
    print("----------")
    print(f"{df_name}:")

    ### Find names and averages
    avg = list(df.iloc[-1])[1:]
    cat = list(df)[1:]
    
    ### Compare differences with critical value
    for m_name, score in zip(cat, avg):
        for n_name, score2 in zip(cat, avg):
            if score-score2 > cd:
                print(f"[{m_name} - {n_name}] exceeds critical distance")



Critical distance = 1.0478
----------
fit_time:
[Random Forest - Logistic Regression] exceeds critical distance
----------
accuracy_score:
[Logistic Regression - Random Forest] exceeds critical distance
----------
f_score:
[Logistic Regression - Random Forest] exceeds critical distance
